# Assignment 3.2

This notebook contains the solution and outputs for **Assignment 3.2**.

## Setup and Imports

In [1]:
from transformers import pipeline
import time

# Shared memory for agent communication
shared_memory = {}

# Planner Agent: Generates a plan for processing the input
class PlannerAgent:
    def __init__(self):
        self.name = "Planner"
    
    def execute(self, input_text):
        plan = [
            f"Summarize the text: {input_text[:100]}...",
            "Answer questions based on the summary"
        ]
        shared_memory["plan"] = plan
        print(f"{self.name}: Generated plan - {plan}")
        return plan

# Summarizer Agent: Summarizes the input text
class SummarizerAgent:
    def __init__(self):  # Corrected method name
        self.name = "Summarizer"
        try:
            self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)  # device=-1 for CPU
            print(f"{self.name}: Successfully initialized summarizer pipeline.")
        except Exception as e:
            print(f"{self.name}: Failed to initialize summarizer pipeline: {e}")
            raise
    
    def execute(self, input_text):
        if not hasattr(self, 'summarizer'):
            raise AttributeError("Summarizer pipeline not initialized.")
        max_input_length = 1024  # BART's max token limit
        truncated_text = input_text[:max_input_length]
        summary = self.summarizer(truncated_text, max_length=150, min_length=50, do_sample=False)[0]["summary_text"]
        shared_memory["summary"] = summary
        print(f"{self.name}: Generated summary - {summary[:100]}...")
        return summary

# Answerer Agent: Answers questions based on summary
class AnswererAgent:
    def __init__(self):
        self.name = "Answerer"
        try:
            self.qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", device=-1)
            print(f"{self.name}: Successfully initialized QA pipeline.")
        except Exception as e:
            print(f"{self.name}: Failed to initialize QA pipeline: {e}")
            raise
    
    def execute(self, question):
        context = shared_memory.get("summary", "")
        if not context:
            return "No summary available to answer the question."
        answer = self.qa_pipeline(question=question, context=context)["answer"]
        print(f"{self.name}: Answered '{question}' - {answer}")
        return answer

# Multi-Agent System
class MultiAgentSystem:
    def __init__(self):
        self.agents = {
            "planner": PlannerAgent(),
            "summarizer": SummarizerAgent(),
            "answerer": AnswererAgent()
        }
    
    def run(self, input_text, question):
        start_time = time.time()
        
        # Step 1: Planner creates a plan
        self.agents["planner"].execute(input_text)
        
        # Step 2: Summarizer processes the input text
        self.agents["summarizer"].execute(input_text)
        
        # Step 3: Answerer responds to the question
        answer = self.agents["answerer"].execute(question)
        
        print(f"Total execution time: {time.time() - start_time:.2f} seconds")
        return answer

# Example usage
if __name__ == "__main__":
    try:
        input_text = """
        Artificial intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving. AI is used in various applications like self-driving cars, medical diagnosis, and virtual assistants.
        """
        question = "What is artificial intelligence?"
        
        system = MultiAgentSystem()
        answer = system.run(input_text, question)
        print(f"Final Answer: {answer}")
    except Exception as e:
        print(f"Error running multi-agent system: {e}")

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Device set to use cpu


Summarizer: Successfully initialized summarizer pipeline.


Device set to use cpu
Your max_length is set to 150, but your input_length is only 91. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)


Answerer: Successfully initialized QA pipeline.
Planner: Generated plan - ['Summarize the text: \n        Artificial intelligence (AI) refers to the simulation of human intelligence in machines tha...', 'Answer questions based on the summary']
Summarizer: Generated summary - Artificial intelligence (AI) refers to the simulation of human intelligence in machines that are pro...
Answerer: Answered 'What is artificial intelligence?' - simulation of human intelligence in machines that are programmed to think like humans
Total execution time: 8.95 seconds
Final Answer: simulation of human intelligence in machines that are programmed to think like humans
